In [1]:
import numpy as np
import pandas as pd

import matplotlib as plt
%matplotlib inline



from sklearn.model_selection import GridSearchCV,StratifiedKFold, train_test_split
kfold = StratifiedKFold(n_splits=5)
from sklearn.metrics import roc_auc_score

from sklearn.metrics import roc_curve,auc

import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import xgboost as xgb

In [2]:
train=pd.read_csv("train.csv")
test_x=pd.read_csv("test.csv")

In [3]:
train.drop(['id'], axis=1, inplace=True)
test=test_x.drop(['id'], axis=1).copy()

In [4]:
train['age'].fillna(-999, inplace=True)
test['age'].fillna(-999, inplace=True)
train['trainee_engagement_rating'].fillna(-999, inplace=True)
test['trainee_engagement_rating'].fillna(-999, inplace=True)

In [5]:
train.head()

,program_id,program_type,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,Y_1,Y,136,150,offline,intermediate,9389,M,Matriculation,3,24.0,5,N,1.0,0
1,T_1,T,131,44,offline,easy,16523,F,High School Diploma,4,26.0,2,N,3.0,1
2,Z_2,Z,120,178,online,easy,13987,M,Matriculation,1,40.0,1,N,2.0,1
3,T_2,T,117,32,offline,easy,13158,F,Matriculation,3,-999.0,4,N,1.0,1
4,V_3,V,131,84,offline,intermediate,10591,F,High School Diploma,1,42.0,2,N,4.0,1


In [6]:
train[['education', 'is_pass']].groupby(['education'], as_index=False).count().sort_values(by='is_pass', ascending=False)

,education,is_pass
1,High School Diploma,34371
3,Matriculation,26169
0,Bachelors,11581
4,No Qualification,514
2,Masters,512


In [7]:
train.difficulty_level.replace(['easy','intermediate','hard','vary hard'],[1,2,3,4], inplace=True)
test.difficulty_level.replace(['easy','intermediate','hard','vary hard'],[1,2,3,4], inplace=True)
train.education.replace(['High School Diploma','Matriculation','Bachelors','No Qualification','Masters'],[1,2,3,4,5], inplace=True)
test.education.replace(['High School Diploma','Matriculation','Bachelors','No Qualification','Masters'],[1,2,3,4,5], inplace=True)
train.difficulty_level=train.difficulty_level.astype('int64')
test.difficulty_level=test.difficulty_level.astype('int64')
train.education=train.education.astype('int64')
test.education=test.education.astype('int64')

In [8]:
train.head()

,program_id,program_type,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,Y_1,Y,136,150,offline,2,9389,M,2,3,24.0,5,N,1.0,0
1,T_1,T,131,44,offline,1,16523,F,1,4,26.0,2,N,3.0,1
2,Z_2,Z,120,178,online,1,13987,M,2,1,40.0,1,N,2.0,1
3,T_2,T,117,32,offline,1,13158,F,2,3,-999.0,4,N,1.0,1
4,V_3,V,131,84,offline,2,10591,F,1,1,42.0,2,N,4.0,1


In [9]:
train.program_id=train.program_id.str.split('_').str[-1]
test.program_id=test.program_id.str.split('_').str[-1]

In [10]:
train.program_id=train.program_id.astype('int64')
test.program_id=test.program_id.astype('int64')

In [11]:
train.head()

,program_id,program_type,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,1,Y,136,150,offline,2,9389,M,2,3,24.0,5,N,1.0,0
1,1,T,131,44,offline,1,16523,F,1,4,26.0,2,N,3.0,1
2,2,Z,120,178,online,1,13987,M,2,1,40.0,1,N,2.0,1
3,2,T,117,32,offline,1,13158,F,2,3,-999.0,4,N,1.0,1
4,3,V,131,84,offline,2,10591,F,1,1,42.0,2,N,4.0,1


In [12]:
train.dtypes

program_id                     int64
program_type                  object
program_duration               int64
test_id                        int64
test_type                     object
difficulty_level               int64
trainee_id                     int64
gender                        object
education                      int64
city_tier                      int64
age                          float64
total_programs_enrolled        int64
is_handicapped                object
trainee_engagement_rating    float64
is_pass                        int64
dtype: object

In [13]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in train.columns.values:
    if train[col].dtypes=='object':
        le.fit(train[col].values)
        train[col]=le.transform(train[col])
for col in test.columns.values:
    if test[col].dtypes=='object':
        le.fit(test[col].values)
        test[col]=le.transform(test[col])

In [14]:
train.head()

,program_id,program_type,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,1,5,136,150,0,2,9389,1,2,3,24.0,5,0,1.0,0
1,1,1,131,44,0,1,16523,0,1,4,26.0,2,0,3.0,1
2,2,6,120,178,1,1,13987,1,2,1,40.0,1,0,2.0,1
3,2,1,117,32,0,1,13158,0,2,3,-999.0,4,0,1.0,1
4,3,3,131,84,0,2,10591,0,1,1,42.0,2,0,4.0,1


In [15]:
train.describe()

,program_id,program_type,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
count,73147.000000,73147.000000,73147.000000,73147.000000,73147.000000,73147.000000,73147.000000,73147.000000,73147.000000,73147.000000,73147.000000,73147.000000,73147.000000,73147.000000,73147.000000
mean,2.362229,3.268227,128.208676,91.079306,0.404719,1.649446,9843.079620,0.533761,1.723488,2.246845,-356.047206,2.583934,0.089819,1.341490,0.695408
std,1.101985,1.817236,6.889679,51.239048,0.490841,0.824570,5703.556093,0.498862,0.801791,1.011587,502.433768,1.240816,0.285924,32.500351,0.460238
min,1.000000,0.000000,117.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,-999.000000,1.000000,0.000000,-999.000000,0.000000
25%,1.000000,1.000000,121.000000,45.000000,0.000000,1.000000,5040.000000,0.000000,1.000000,1.000000,-999.000000,2.000000,0.000000,1.000000,0.000000
50%,2.000000,3.000000,131.000000,90.000000,0.000000,1.000000,9641.000000,1.000000,2.000000,2.000000,27.000000,2.000000,0.000000,2.000000,1.000000
75%,3.000000,5.000000,134.000000,135.000000,1.000000,2.000000,14552.000000,1.000000,2.000000,3.000000,43.000000,3.000000,0.000000,4.000000,1.000000
max,4.000000,6.000000,136.000000,187.000000,1.000000,4.000000,20098.000000,1.000000,5.000000,4.000000,63.000000,14.000000,1.000000,5.000000,1.000000


In [16]:
Y_train = train["is_pass"]
X_train = train.drop("is_pass", axis=1)


X_train.shape, Y_train.shape

((73147, 14), (73147,))

In [157]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
X_train=scaler.fit_transform(X_train)
test=scaler.fit_transform(test)

In [17]:
sm=SMOTE()
X_train, Y_train=sm.fit_sample(X_train, Y_train)

In [18]:
train_x,val_x,train_y,val_y = train_test_split(X_train, Y_train, test_size = 0.10, random_state= 1984)

In [19]:
train_x.shape,val_x.shape,train_y.shape,val_y.shape


((91560, 14), (10174, 14), (91560,), (10174,))

In [20]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=1200,
                   max_depth=8,
                   learning_rate=0.15,
                   subsample=0.85,
                   colsample_bytree=0.85,
                   gamma=0,
                   seed=7
                  )


In [21]:
xgb_model.fit(train_x, train_y)
y_pred = xgb_model.predict_proba(train_x)
roc_auc_score(train_y, y_pred[:,1])

0.99436357044059531

In [22]:
y_pred1 = xgb_model.predict_proba(val_x)

In [23]:
roc_auc_score(val_y, y_pred1[:,1])

0.91745181594125458

In [25]:
from xgboost import XGBClassifier

xgb_model1 = XGBClassifier(n_estimators=1500,
                   max_depth=7,
                   learning_rate=0.15,
                   subsample=0.85,
                   colsample_bytree=0.85,
                   gamma=0.01,
                   seed=7
                  )
xgb_model1.fit(train_x, train_y)
y_pred = xgb_model1.predict_proba(train_x)
roc_auc_score(train_y, y_pred[:,1])

0.99020183051273158

In [26]:
y_pred2 = xgb_model1.predict_proba(val_x)
roc_auc_score(val_y, y_pred2[:,1])

0.91630665039250903

In [27]:
roc_auc_score(val_y, (y_pred2[:,1]+y_pred1[:,1])*0.5)

0.91808443363210512

In [30]:
xgb_model1.fit(X_train, Y_train)




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.85, gamma=0.01, learning_rate=0.15,
       max_delta_step=0, max_depth=7, min_child_weight=1, missing=None,
       n_estimators=1500, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=7, silent=True,
       subsample=0.85)

In [31]:
xgb_model.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.85, gamma=0, learning_rate=0.15,
       max_delta_step=0, max_depth=8, min_child_weight=1, missing=None,
       n_estimators=1200, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=7, silent=True,
       subsample=0.85)

In [28]:
xgb_model2 = XGBClassifier(n_estimators=1500,
                   max_depth=8,
                   learning_rate=0.15,
                   subsample=0.8,
                   colsample_bytree=0.8,
                   gamma=0.05,
                   seed=8
                  )
xgb_model2.fit(X_train, Y_train)
y_pred = xgb_model2.predict_proba(X_train)
roc_auc_score(Y_train, y_pred[:,1])

0.99605942130500591

In [29]:
y_pred3 = xgb_model2.predict_proba(val_x)
roc_auc_score(val_y, y_pred3[:,1])

0.99549678226759863

In [34]:
y_pre1 = xgb_model.predict_proba(test.values)

In [33]:
y_pre2 = xgb_model1.predict_proba(test.values)

In [36]:
y_pre3 = xgb_model2.predict_proba(test.values)

In [37]:
submission = pd.DataFrame({
        "id": test_x["id"],
        "is_pass": (y_pre1[:,1]+y_pre2[:,1]+y_pre3[:,1])*(1/3)
    })
submission.to_csv('submission26.csv', index=False)

In [ ]:
# got result of .8159 on public leaderboard and .8167 on private leaderboard